In [1]:
pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Successfully uninstalled torchvision-0.20.1
Found existing installation: torchaudio 2.5.1
Uninstalling torchaudio-2.5.1:
  Successfully uninstalled torchaudio-2.5.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch torchvision torchaudio

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached torchaudio-2.5.1-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
Using cached torch-2.5.1-cp311-cp311-win_amd64.whl (203.1 MB)
Using cached torchvision-0.20.1-cp311-cp311-win_amd64.whl (1.6 MB)
Using cached torchaudio-2.5.1-cp311-cp311-win_amd64.whl (2.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import requests
from ultralytics import YOLO

In [5]:
# Telegram bot setup
TELEGRAM_BOT_TOKEN = '7797273300:AAEt21FaUKyc4seiHFLf1p3eNnHWuc72nnM'
TELEGRAM_CHAT_ID = '5830227450'

In [6]:
def get_location():
    """Fetch the current location based on IP address using ipinfo.io."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        loc = data.get("loc")
        if loc:
            latitude, longitude = map(float, loc.split(','))
            print(f"Current Location: Latitude={latitude}, Longitude={longitude}")
            return latitude, longitude
        else:
            print("Location information not available.")
            return None, None
    except Exception as e:
        print(f"Error fetching location: {e}")
        return None, None

In [7]:
def send_location_to_telegram():
    latitude, longitude = get_location()
    if latitude and longitude:
        token = '7797273300:AAEt21FaUKyc4seiHFLf1p3eNnHWuc72nnM'
        chat_id = '5830227450'
        url = f'https://api.telegram.org/bot{token}/sendLocation'
        response = requests.post(url, data={
            'chat_id': chat_id,
            'latitude': latitude,
            'longitude': longitude
        })
        if response.status_code == 200:
            print("Location sent successfully!")
        else:
            print("Failed to send location:", response.json())
    else:
        print("Could not fetch location.")

In [8]:
def send_telegram_alert(image_path, detected_objects):
    """Send a warning message and both original and black-and-white images to Telegram"""

    detected_objects_str = ''.join(detected_objects)
    message = f"Alert! Detected objects: {detected_objects_str}"
    
    # Corrected URLs with both bot token and chat ID
    bot_token = "7797273300:AAEt21FaUKyc4seiHFLf1p3eNnHWuc72nnM"
    chat_id = "5830227450"
    text_url = f'https://api.telegram.org/bot7797273300:AAEt21FaUKyc4seiHFLf1p3eNnHWuc72nnM/sendMessage'
    photo_url = f'https://api.telegram.org/bot7797273300:AAEt21FaUKyc4seiHFLf1p3eNnHWuc72nnM/sendPhoto'

    # Send text message
    requests.post(text_url, data={'chat_id': 5830227450, 'text': message})
    
    # Read the original image
    original_image = cv2.imread(image_path)

    # Convert to high-quality black-and-white (grayscale with contrast enhancement)
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    high_quality_bw = cv2.equalizeHist(gray_image)
    
    # Save the black-and-white version temporarily
    bw_image_path = "high_quality_bw.jpg"
    cv2.imwrite(bw_image_path, high_quality_bw)

    
    # Send original image to Telegram
    with open(image_path, 'rb') as img:
        requests.post(photo_url, data={'chat_id': 5830227450, 'caption': "Original Image"}, files={'photo': img})
    
    # Send black-and-white image to Telegram
    with open(bw_image_path, 'rb') as bw_img:
        requests.post(photo_url, data={'chat_id': 5830227450, 'caption': "High-Quality Black and White Image"}, files={'photo': bw_img})

    




In [9]:
model = YOLO("yolo-Weights/yolov8n.pt")

In [10]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush","toy"]

In [11]:
def detect_objects():
    cap = cv2.VideoCapture(0)  
    detected = set()  
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame,conf=0.8)
        current_detections = set()  # Store current detections for this frame
        
        # Iterate through the detected results
        for result in results:
            boxes = result.boxes  # Get the boxes from the result
            for box in boxes:  # Loop through each box
                label = int(box.cls.item())  # Convert class ID tensor to int
                confidence = box.conf.item()  # Convert confidence tensor to float
                if confidence > 0.5:  # Consider only detections with confidence > 0.5
                    detected_object = classNames[label]
                    current_detections.add(detected_object)  # Add the detected class name
                    
                    # Draw bounding boxes on frame
                    x1, y1, x2, y2 = box.xyxy.numpy()[0]  # Get the bounding box coordinates
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                    cv2.putText(frame, f"{detected_object}: {confidence:.2f}", 
                                (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    
                    # Check if the object is newly detected
                    if detected_object not in detected:
                        # Save frame with detection and send to Telegram
                        cv2.imwrite(f"{detected_object}_detected.jpg", frame)
                        send_telegram_alert(f"{detected_object}_detected.jpg", detected_object)
                        send_location_to_telegram()
                        detected.add(detected_object)  # Update detected set
        cv2.imshow("Object Detection", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()



In [27]:
if __name__ == "__main__":
    detect_objects()


0: 480x640 1 person, 200.2ms
Speed: 10.0ms preprocess, 200.2ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
Current Location: Latitude=25.5941, Longitude=85.1356
Location sent successfully!

0: 480x640 1 person, 221.1ms
Speed: 3.0ms preprocess, 221.1ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 213.4ms
Speed: 5.0ms preprocess, 213.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 223.0ms
Speed: 6.1ms preprocess, 223.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 215.1ms
Speed: 5.0ms preprocess, 215.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 292.7ms
Speed: 5.0ms preprocess, 292.7ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 322.6ms
Speed: 5.0ms preprocess, 322.6ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person,